# 2.4 微积分

在计算y关于x求导之前，我们需要一个地方来存储梯度
假设相对y=2*x^T*x关于向量x求导，首先我们要创建一个变量x并为其分配一个初始值

In [2]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [3]:
x.requires_grad_(True)# 等价于x=torch。arange（4.0，requires_grad=True）
x.grad# 默认值为None

现在计算y

In [4]:
y = 2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

x是一个长度为4的向量，计算x和x的点积，得到了我们赋值给y的标量输出。再通过调用反向传播函数来自动计算y关于每个分量的梯度，并且打印这些梯度。

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

函数y=2*x^T*x关于x的梯度应为4x，验证：

In [6]:
x.grad==4*x

tensor([True, True, True, True])

现在计算x的另一个函数

In [7]:
# 在默认情况下，pytorch会累计梯度，我们需要清楚之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## 非标量变量的反向传播
当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。对于高阶和高维的y和x。求导的结果可以是一个高阶张量

In [8]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
x.grad.zero_()
y = x*x# 矩阵
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()# sum中axis默认为0
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外（之后需要再回看，按我自己的理解就是u是z的系数，其实也就是x的梯度（斜率））

In [15]:
x.grad.zero_()# 梯度清零
y = x*x
u = y.detach()# 视作一个常数
z = u*x
z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [16]:
x.grad.zero_()
y.sum().backward()# 这里仍然是y=x*x的矩阵，其实2*x就是y对x的偏导数
x.grad == 2*x

tensor([True, True, True, True])

即使构建函数的计算图需要通过python控制流（条件、循环、或任意函数调用），我们仍然可以计算得到的变量的梯度

In [19]:
def f(a):
    b = a * 2
    while b.norm() < 1000: # norm是向量元素平方和的平方根
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(),requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a 

tensor(True)